In [1]:
import pandas as pd
import numpy as np
import os, sys, math

In [47]:
from rdkit import Chem
from rdkit.Chem import *
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.rdMolDescriptors import *
from rdkit import DataStructs
from rdkit.Chem import AllChem

# 

# Read Drug-Gene-Cancer Type Interactions

In [5]:
drugcentral_df = pd.read_csv('../data/DrugCentral/processed_data/drug.target.interaction.fda.cosmic.cancer.type.tsv', sep = '\t')

In [6]:
drugcentral_df.head()

,DRUG_NAME,STRUCT_ID,TARGET_NAME,TARGET_CLASS,ACCESSION,GENE,SWISSPROT,ACT_VALUE,ACT_UNIT,ACT_TYPE,...,TDL,ORGANISM,Driver_Gene,FDA_Approved,EMA_Approved,PMDA_Approved,Bladder,Colon,NSCLC,Ovarian
0,levobupivacaine,4,Potassium voltage-gated channel subfamily H me...,Ion channel,Q12809,KCNH2,KCNH2_HUMAN,4.890,NaN,IC50,...,Tclin,Homo sapiens,No,Yes,No,No,No,No,No,No
1,levobupivacaine,4,Sodium channel protein type 1 subunit alpha,Ion channel,P35498,SCN1A,SCN1A_HUMAN,5.790,NaN,IC50,...,Tclin,Homo sapiens,No,Yes,No,No,No,No,No,No
2,levobupivacaine,4,Sodium channel protein type 4 subunit alpha,Ion channel,P35499,SCN4A,SCN4A_HUMAN,NaN,NaN,NaN,...,Tclin,Homo sapiens,No,Yes,No,No,No,No,No,No
3,levobupivacaine,4,Prostaglandin E2 receptor EP1 subtype,GPCR,P34995,PTGER1,PE2R1_HUMAN,NaN,NaN,NaN,...,Tclin,Homo sapiens,No,Yes,No,No,No,No,No,No
4,levobupivacaine,4,Cytochrome P450 2D6,Enzyme,P10635,CYP2D6,CP2D6_HUMAN,6.707,NaN,IC50,...,Tclin,Homo sapiens,No,Yes,No,No,No,No,No,No


In [8]:
drugcentral_df.columns.to_list()

['DRUG_NAME',
 'STRUCT_ID',
 'TARGET_NAME',
 'TARGET_CLASS',
 'ACCESSION',
 'GENE',
 'SWISSPROT',
 'ACT_VALUE',
 'ACT_UNIT',
 'ACT_TYPE',
 'ACT_COMMENT',
 'ACT_SOURCE',
 'RELATION',
 'MOA',
 'MOA_SOURCE',
 'ACT_SOURCE_URL',
 'MOA_SOURCE_URL',
 'ACTION_TYPE',
 'TDL',
 'ORGANISM',
 'Driver_Gene',
 'FDA_Approved',
 'EMA_Approved',
 'PMDA_Approved',
 'Bladder',
 'Colon',
 'NSCLC',
 'Ovarian']

# 

# Read Scores

In [15]:
score_df = pd.read_csv('../data/DrugCentral/processed_data/drug.target.interaction.scores.tsv', sep = '\t')
score_df.head()

,DRUG_NAME,nGENE,BLADDER_CPS,BLADDER_CGPS,COLON_CPS,COLON_CGPS,NSCLC_CPS,NSCLC_CGPS,OVARIAN_CPS,OVARIAN_CGPS
0,(S)-nicardipine,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,(S)-nitrendipine,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Choline C-11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Florbetapir F-18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Flutemetamol (18F),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
drug_gene_count_df = score_df[['DRUG_NAME', 'nGENE']].drop_duplicates()

# 

# Read SMILES Strings

In [169]:
smiles_df = pd.read_csv('../data/DrugCentral/raw_data/structures.smiles.tsv', sep = '\t')
smiles_df.head()

,SMILES,InChI,InChIKey,ID,INN,CAS_RN
0,CNC(=O)C1=C(C=C(C=C1)C2=NN3C(=CN=C3N=C2)CC4=CC...,InChI=1S/C23H17FN6O/c1-25-22(31)18-6-5-16(11-1...,LIOLIMKSCNQPLV-UHFFFAOYSA-N,5392,capmatinib,1029712-80-8
1,CC(C)(COC1=CN2C(=C(C=N2)C#N)C(=C1)C3=CN=C(C=C3...,"InChI=1S/C29H31N7O3/c1-29(2,37)18-39-24-9-25(2...",XIIOFHFUYBLOLW-UHFFFAOYSA-N,5393,selpercatinib,2152628-33-4
2,CCN1C2=CC(=NC=C2C=C(C1=O)C3=CC(=C(C=C3Br)F)NC(...,InChI=1S/C24H21BrFN5O2/c1-3-31-21-12-22(27-2)2...,CEFJVGZHQAGLHS-UHFFFAOYSA-N,5394,ripretinib,1442472-39-0
3,C[C@]12CC[C@H]3[C@H]([C@@H]1C[C@H]([C@@H]2O)[1...,InChI=1S/C18H23FO2/c1-18-7-6-13-12-5-3-11(20)8...,KDLLNMRYZGUVMA-ZYMZXAKXSA-N,5395,fluoroestradiol F 18,94153-53-4
4,C1=CC2=C(C=C1C3=CN=C(C=C3)[18F])NC4=C2C=NC=C4,InChI=1S/C16H10FN3/c17-16-4-2-11(8-19-16)10-1-...,GETAAWDSFUCLBS-SJPDSGJFSA-N,5396,flortaucipir F 18,1522051-90-6


In [171]:
subset_smiles_df = smiles_df[['SMILES', 'ID']].drop_duplicates()

In [173]:
subset_smiles_df

,SMILES,ID
0,CNC(=O)C1=C(C=C(C=C1)C2=NN3C(=CN=C3N=C2)CC4=CC...,5392
1,CC(C)(COC1=CN2C(=C(C=N2)C#N)C(=C1)C3=CN=C(C=C3...,5393
2,CCN1C2=CC(=NC=C2C=C(C1=O)C3=CC(=C(C=C3Br)F)NC(...,5394
3,C[C@]12CC[C@H]3[C@H]([C@@H]1C[C@H]([C@@H]2O)[1...,5395
4,C1=CC2=C(C=C1C3=CN=C(C=C3)[18F])NC4=C2C=NC=C4,5396
...,...,...
4094,COC(=O)[C@@H]([C@H]1CCCCN1C(=O)OC[N+]2=CC=CC(=...,5448
4095,C[C@]12CC[C@H]3[C@H]([C@@H]1[C@H]([C@H]([C@@H]...,5450
4096,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)C1=C...,5458
4097,CCN1CCN(CC1)C2=CC=C(C=C2)NC3=CC(=NC=N3)N(C)C(=...,5459


# 

# Merge Drug-Gene-Cancer Type Interactions with SMILES Strings

In [29]:
drugcentral_smile_df = pd.merge(drugcentral_df, subset_smiles_df, left_on = 'STRUCT_ID', right_on = 'ID', how = 'left')
drugcentral_smile_df

,DRUG_NAME,STRUCT_ID,TARGET_NAME,TARGET_CLASS,ACCESSION,GENE,SWISSPROT,ACT_VALUE,ACT_UNIT,ACT_TYPE,...,Driver_Gene,FDA_Approved,EMA_Approved,PMDA_Approved,Bladder,Colon,NSCLC,Ovarian,SMILES,ID
0,levobupivacaine,4,Potassium voltage-gated channel subfamily H me...,Ion channel,Q12809,KCNH2,KCNH2_HUMAN,4.890,NaN,IC50,...,No,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0
1,levobupivacaine,4,Sodium channel protein type 1 subunit alpha,Ion channel,P35498,SCN1A,SCN1A_HUMAN,5.790,NaN,IC50,...,No,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0
2,levobupivacaine,4,Sodium channel protein type 4 subunit alpha,Ion channel,P35499,SCN4A,SCN4A_HUMAN,NaN,NaN,NaN,...,No,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0
3,levobupivacaine,4,Prostaglandin E2 receptor EP1 subtype,GPCR,P34995,PTGER1,PE2R1_HUMAN,NaN,NaN,NaN,...,No,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0
4,levobupivacaine,4,Cytochrome P450 2D6,Enzyme,P10635,CYP2D6,CP2D6_HUMAN,6.707,NaN,IC50,...,No,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19373,samidorphan,5460,Delta-type opioid receptor,GPCR,P41143,OPRD1,OPRD_HUMAN,8.590,NaN,Ki,...,No,Yes,No,No,No,No,No,No,C1CC1CN2CCC34CC(=O)CCC3(C2CC5=C4C(=C(C=C5)C(=O...,5460.0
19374,sotorasib,5461,GTPase KRas,Enzyme,P01116,KRAS,RASK_HUMAN,7.030,NaN,IC50,...,Yes,Yes,Yes,Yes,No,No,No,No,C[C@H](C(C)C)[C@]1(CC[C@@]2([C@H]3CC[C@H]4[C@]...,5461.0
19375,ibrexafungerp,5462,"Beta-1,3-glucan synthase catalytic subunit 1",Enzyme,O13428,GSC1,O13428_CANAX,8.350,NaN,IC50,...,No,Yes,No,No,No,No,No,No,C[C@H](C(C)C)[C@]1(CC[C@@]2([C@H]3CC[C@H]4[C@]...,5462.0
19376,ibrexafungerp,5462,"1,3-Beta-D-glucan-UDP glucosyltransferase",Enzyme,Q6FTN8,FKS1,Q6FTN8_CANGA,7.830,NaN,IC50,...,No,Yes,No,No,No,No,No,No,C[C@H](C(C)C)[C@]1(CC[C@@]2([C@H]3CC[C@H]4[C@]...,5462.0


In [31]:
drugcentral_smile_count_df = pd.merge(drugcentral_smile_df, drug_gene_count_df, on = 'DRUG_NAME', how = 'left')

In [33]:
drugcentral_smile_count_df

,DRUG_NAME,STRUCT_ID,TARGET_NAME,TARGET_CLASS,ACCESSION,GENE,SWISSPROT,ACT_VALUE,ACT_UNIT,ACT_TYPE,...,FDA_Approved,EMA_Approved,PMDA_Approved,Bladder,Colon,NSCLC,Ovarian,SMILES,ID,nGENE
0,levobupivacaine,4,Potassium voltage-gated channel subfamily H me...,Ion channel,Q12809,KCNH2,KCNH2_HUMAN,4.890,NaN,IC50,...,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0,8.0
1,levobupivacaine,4,Sodium channel protein type 1 subunit alpha,Ion channel,P35498,SCN1A,SCN1A_HUMAN,5.790,NaN,IC50,...,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0,8.0
2,levobupivacaine,4,Sodium channel protein type 4 subunit alpha,Ion channel,P35499,SCN4A,SCN4A_HUMAN,NaN,NaN,NaN,...,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0,8.0
3,levobupivacaine,4,Prostaglandin E2 receptor EP1 subtype,GPCR,P34995,PTGER1,PE2R1_HUMAN,NaN,NaN,NaN,...,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0,8.0
4,levobupivacaine,4,Cytochrome P450 2D6,Enzyme,P10635,CYP2D6,CP2D6_HUMAN,6.707,NaN,IC50,...,Yes,No,No,No,No,No,No,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19373,samidorphan,5460,Delta-type opioid receptor,GPCR,P41143,OPRD1,OPRD_HUMAN,8.590,NaN,Ki,...,Yes,No,No,No,No,No,No,C1CC1CN2CCC34CC(=O)CCC3(C2CC5=C4C(=C(C=C5)C(=O...,5460.0,3.0
19374,sotorasib,5461,GTPase KRas,Enzyme,P01116,KRAS,RASK_HUMAN,7.030,NaN,IC50,...,Yes,Yes,Yes,No,No,No,No,C[C@H](C(C)C)[C@]1(CC[C@@]2([C@H]3CC[C@H]4[C@]...,5461.0,1.0
19375,ibrexafungerp,5462,"Beta-1,3-glucan synthase catalytic subunit 1",Enzyme,O13428,GSC1,O13428_CANAX,8.350,NaN,IC50,...,Yes,No,No,No,No,No,No,C[C@H](C(C)C)[C@]1(CC[C@@]2([C@H]3CC[C@H]4[C@]...,5462.0,3.0
19376,ibrexafungerp,5462,"1,3-Beta-D-glucan-UDP glucosyltransferase",Enzyme,Q6FTN8,FKS1,Q6FTN8_CANGA,7.830,NaN,IC50,...,Yes,No,No,No,No,No,No,C[C@H](C(C)C)[C@]1(CC[C@@]2([C@H]3CC[C@H]4[C@]...,5462.0,3.0


# 

# Standardize SMILES Strings and Convert into Molecules

In [175]:
# PandasTools.AddMoleculeColumnToFrame(smiles_df, smilesCol='SMILES')

In [177]:
smiles_list = smiles_df['SMILES']
mol_list = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

[10:40:06] WARNING: not removing hydrogen atom without neighbors


In [178]:
smiles_df['Molecule'] = mol_list

In [181]:
new_smiles_list = [Chem.MolToSmiles(mol) for mol in mol_list]
smiles_df['SMILES_String'] = new_smiles_list

# 

# Create Fingerprints from Molecules

In [53]:
fpgen = AllChem.GetRDKitFPGenerator()
fps = [fpgen.GetFingerprint(x) for x in mol_list]
smiles_df['Fingerprint'] = fps

# 

# Generate Matrix for All Drug Pairs

In [193]:
def create_pairs_matrix(input_list):
    x_cols = []
    y_cols = []
    
    matrix = []
    
    for i in input_list:
        for j in input_list:
            matrix.append([i, j])

            x_cols.append(i)
            y_cols.append(j)

    return matrix, x_cols, y_cols

In [195]:
smiles_combination_matrix, x_cols, y_cols = create_pairs_matrix(new_smiles_list)
len(x_cols), len(y_cols), len(smiles_combination_matrix)

(16801801, 16801801, 16801801)

In [213]:
drug_combination_df = pd.DataFrame()
drug_combination_df['Molecule1'] = x_cols
drug_combination_df['Molecule2'] = y_cols
drug_combination_df = drug_combination_df.drop_duplicates()

In [215]:
drug_combination_df

,Molecule1,Molecule2
0,CNC(=O)c1ccc(-c2cnc3ncc(Cc4ccc5ncccc5c4)n3n2)cc1F,CNC(=O)c1ccc(-c2cnc3ncc(Cc4ccc5ncccc5c4)n3n2)cc1F
1,CNC(=O)c1ccc(-c2cnc3ncc(Cc4ccc5ncccc5c4)n3n2)cc1F,COc1ccc(CN2C3CC2CN(c2ccc(-c4cc(OCC(C)(C)O)cn5n...
2,CNC(=O)c1ccc(-c2cnc3ncc(Cc4ccc5ncccc5c4)n3n2)cc1F,CCn1c(=O)c(-c2cc(NC(=O)Nc3ccccc3)c(F)cc2Br)cc2...
3,CNC(=O)c1ccc(-c2cnc3ncc(Cc4ccc5ncccc5c4)n3n2)cc1F,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...
4,CNC(=O)c1ccc(-c2cnc3ncc(Cc4ccc5ncccc5c4)n3n2)cc1F,[18F]c1ccc(-c2ccc3c(c2)[nH]c2ccncc23)cn1
...,...,...
16801796,NC(=O)c1ccc2c(c1O)C13CCN(CC4CC4)C(C2)C1(O)CCC(...,COC(=O)[C@H](c1ccccc1)[C@H]1CCCCN1C(=O)OC[n+]1...
16801797,NC(=O)c1ccc2c(c1O)C13CCN(CC4CC4)C(C2)C1(O)CCC(...,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1[C@@...
16801798,NC(=O)c1ccc2c(c1O)C13CCN(CC4CC4)C(C2)C1(O)CCC(...,O=C(O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc([1...
16801799,NC(=O)c1ccc2c(c1O)C13CCN(CC4CC4)C(C2)C1(O)CCC(...,CCN1CCN(c2ccc(Nc3cc(N(C)C(=O)Nc4c(Cl)c(OC)cc(O...


# 

# Calculate Drug Similarity Score 

## Similarity Based on Tanimoto Score: 0 - 1 

In [149]:
def calculate_similarity_score(mol1, mol2):
    return DataStructs.TanimotoSimilarity(mol1, mol2)

In [ ]:
def convert_smiles_to_molecules_and_calculate_score(smiles_list):

    mol_list = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

    fpgen = AllChem.GetRDKitFPGenerator()
    fps = [fpgen.GetFingerprint(x) for x in mol_list]
    
    similarity = calculate_similarity_score(fps[0], fps[1])
   
    return similarity

In [221]:
def create_score_pairs_matrix(df):
    similarity_scores = []
    
    for index, row in df.iterrows():
        smiles_list = [row['Molecule1'], row['Molecule2']]
        mol_list = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
        
        fpgen = AllChem.GetRDKitFPGenerator()
        fps = [fpgen.GetFingerprint(x) for x in mol_list]
        
        similarity = calculate_similarity_score(fps[0], fps[1])
        similarity_scores.append(similarity)
            
    return similarity_scores

In [ ]:
similarity_scores = create_score_pairs_matrix(drug_combination_df)

[10:46:44] WARNING: not removing hydrogen atom without neighbors
[10:46:53] WARNING: not removing hydrogen atom without neighbors
[10:47:01] WARNING: not removing hydrogen atom without neighbors
[10:47:08] WARNING: not removing hydrogen atom without neighbors
[10:47:14] WARNING: not removing hydrogen atom without neighbors
[10:47:25] WARNING: not removing hydrogen atom without neighbors
[10:47:35] WARNING: not removing hydrogen atom without neighbors
[10:47:41] WARNING: not removing hydrogen atom without neighbors
[10:47:47] WARNING: not removing hydrogen atom without neighbors
[10:47:55] WARNING: not removing hydrogen atom without neighbors
[10:48:05] WARNING: not removing hydrogen atom without neighbors
[10:48:13] WARNING: not removing hydrogen atom without neighbors
[10:48:22] WARNING: not removing hydrogen atom without neighbors
[10:48:27] WARNING: not removing hydrogen atom without neighbors
[10:48:37] WARNING: not removing hydrogen atom without neighbors
[10:48:42] WARNING: not r

# 

# Write Drug Combinations with Similarity Score

In [ ]:
drug_combination_df['Tanimoto_Score'] = similarity_scores

In [ ]:
drug_combination_df.to_csv('drug.drug.score.csv', index = False)